In [27]:
# Initialize Otter
import otter
grader = otter.Notebook("proj1.ipynb")

# Project 1 - SQL
## Due Date: Friday, September 23rd, 11:59pm

In this project we will be working with SQL on the IMDB database.

## Objectives

- Explore and extract relevant information from database with SQL functions
- Perform data cleaning and transformation using string functions and regex
- Use the cleaned data to run insightful analysis using joins, aggregations, and window functions

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

Each coding question has **both public tests and hidden tests**. Roughly 50% of your grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests. In addition, there are two free-response questions that will be manually graded.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

|Question|Points|
|---|---|
|0|1|
|1a|1|
|1b|2|
|1c|1|
|2ai|1|
|2aii|3|
|2bi|2|
|2bii|2|
|2biii|2|
|2biv|1|
|3a|2|
|3b|2|
|3c|1|
|4|2|
|**Total**|23|

In [28]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Magic Commands

Before getting started, read about line magic (``%``) and cell magic (``%%``) [here](https://www.tutorialspoint.com/jupyter/ipython_magic_commands.htm). These commands will be used extensively in this project and future projects to aid us in running SQL queries.

Here are the two ways of writing a SQL query and storing the query result into a variable:
- Single-line magic: ``result = %sql SELECT * FROM table ...``
- Mutli-line cell magic:

``%%sql result << 
SELECT * 
FROM table ...``

# Getting Connected
We are going to be using the `ipython-sql` library to connect our notebook to a PostgreSQL database server on your jupyterhub account. The next cell should do the trick; you should not see any error messages after it completes.

In [29]:
# The first time you are running this cell, you may need to run the following line as: %load_ext sql 
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

Now we need to unzip the data. This only needs to be done once.

In [30]:
!unzip -u data/imdbdb.zip -d data/

Archive:  data/imdbdb.zip


We will use PostgreSQL commands to create a database and import our data into it. Run the following cell to do this. The line `%sql \l` will display the databases made and you should see a database called `imdb`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [31]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb'
!psql -h localhost -c 'CREATE DATABASE imdb' 
!psql -h localhost -d imdb -f data/imdbdb.sql
%sql \l

ERROR:  database "imdb" is being accessed by other users
DETAIL:  There is 1 other session using the database.
ERROR:  database "imdb" already exists
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
psql:data/imdbdb.sql:31: ERROR:  relation "actor_sample" already exists
psql:data/imdbdb.sql:34: ERROR:  role "postgres" does not exist
psql:data/imdbdb.sql:45: ERROR:  relation "cast_sample" already exists
psql:data/imdbdb.sql:48: ERROR:  role "postgres" does not exist
psql:data/imdbdb.sql:59: ERROR:  relation "movie_info_sample" already exists
psql:data/imdbdb.sql:62: ERROR:  role "postgres" does not exist
psql:data/imdbdb.sql:71: ERROR:  relation "cut1" already exists
psql:data/imdbdb.sql:74: ERROR:  role "postgres" does not exist
psql:data/imdbdb.sql:83: ERROR:  relation "info_type" already exists
psql:data/imdbdb.sql:86: ERROR:  role "imdb" does not exist
psql:data/imdbdb.sql:96: ERROR:  relation "movie_sample" already exists
psql:data/imdbdb.sql:99: ERR

[('imdb', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('jovyan', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('postgres', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('template0',
  'jovyan',
  'UTF8',
  'en_US.utf8',
  'en_US.utf8',
  '=c/jovyan\njovyan=CTc/jovyan'),
 ('template1',
  'jovyan',
  'UTF8',
  'en_US.utf8',
  'en_US.utf8',
  '=c/jovyan\njovyan=CTc/jovyan')]

Now let's connect to the new database we just created! There should be no errors after running the following cell.

In [32]:
%sql postgresql://jovyan@127.0.0.1:5432/imdb

To make sure things are working, let's fetch 10 rows from one of our tables.

In [33]:
%%sql
SELECT * 
  FROM cast_sample
LIMIT 10

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


[(708, 235, 2345369, 1),
 (721, 241, 2504309, 1),
 (789, 264, 2156734, 1),
 (875, 299, 1954994, 1),
 (888, 302, 765037, 1),
 (889, 302, 765172, 1),
 (898, 306, 291387, 1),
 (899, 306, 1477434, 1),
 (931, 324, 824119, 1),
 (1936, 543, 1754068, 1)]

Run the following cell for grading purposes.

In [34]:
!mkdir -p results

## Table Descriptions

We are working with the IMDB database. This database is huge and has a lot of information that we have pared down for this project. 

- **actor_sample**: information about the actors including id, name, and gender
- **cast_sample**: each person on the cast of each movie gets a row including cast id, person's id (actor_sample.id), movie id, and role id
- **movie_sample**: sample of movies the actors have been in including movie's id, title, and the production year
- **movie_info_sample**: this table originally had a lot of information for each movie (take a look at info_type to see the information available) but we have dropped some information to make it a bit easier to manage. This table includes movie info's id, movie id, info type id, and the info itself
- **info_type**: reference table to match each info type id to the description of the type of information
- **role_type**: reference table for cast_sample to match role id to the description of the role

To make these tables smaller in your snapshot of IMDB, we took a random sample of actors from the full database, and included their corresponding movies and cast info. You will learn how to do sampling in SQL below.

Let's look at metadata about the tables. Many database clilent applications (like psql) and connectivity libraries offer some convenience commands for exploring metadata. We can use the psql's meta-commands (also called "backslash commands") from Jupyter directly.

In [35]:
%sql \d

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.


[('public', 'actor_sample', 'table', 'jovyan'),
 ('public', 'cast_sample', 'table', 'jovyan'),
 ('public', 'cut1', 'view', 'jovyan'),
 ('public', 'info_type', 'table', 'jovyan'),
 ('public', 'movie_info_sample', 'table', 'jovyan'),
 ('public', 'movie_sample', 'table', 'jovyan'),
 ('public', 'number_movies', 'view', 'jovyan'),
 ('public', 'role_type', 'table', 'jovyan')]

You can get quick help on psql meta-commands via \?:

In [36]:
%sql \?

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
33 rows affected.


[('\\! [command]', 'Pass commands to shell.'),
 ('\\?', 'Show Commands.'),
 ('\\copy [tablename] to/from [filename]',
  'Copy data between a file and a table.'),
 ('\\d[+] [pattern]', 'List or describe tables, views and sequences.'),
 ('\\dD[+] [pattern]', 'List or describe domains.'),
 ('\\dE[+] [pattern]', 'List foreign tables.'),
 ('\\dF[+] [pattern]', 'List text search configurations.'),
 ('\\dT[S+] [pattern]', 'List data types'),
 ('\\db[+] [pattern]', 'List tablespaces.'),
 ('\\ddp [pattern]', 'Lists default access privilege settings.'),
 ('\\df[+] [pattern]', 'List functions.'),
 ('\\di[+] [pattern]', 'List indexes.'),
 ('\\dm[+] [pattern]', 'List materialized views.'),
 ('\\dn[+] [pattern]', 'List schemas.'),
 ('\\dp [pattern]', 'List roles.'),
 ('\\ds[+] [pattern]', 'List sequences.'),
 ('\\dt[+] [pattern]', 'List tables.'),
 ('\\du[+] [pattern]', 'List roles.'),
 ('\\dv[+] [pattern]', 'List views.'),
 ('\\dx[+] [pattern]', 'List extensions.'),
 ('\\e [file]', 'Edit the query with external editor.'),
 ('\\h', 'Show SQL syntax and help.'),
 ('\\l[+] [pattern]', 'List databases.'),
 ('\\n[+] [name] [param1 param2 ...]', 'List or execute named queries.'),
 ('\\nd [name]', 'Delete a named query.'),
 ('\\np name_pattern', 'Print a named query.'),
 ('\\ns name query', 'Save a named query.'),
 ('\\pager [command]', 'Set PAGER. Print the query results via PAGER.'),
 ('\\pset [key] [value]', 'A limited version of traditional \\pset'),
 ('\\sf[+] FUNCNAME', "Show a function's definition."),
 ('\\timing', 'Toggle timing of commands.'),
 ('\\watch [sec=2]', 'Execute query every `sec` seconds.'),
 ('\\x', 'Toggle expanded output.')]

There is a more "native SQL" way to look at metadata. SQL stores its metadata in tables -- essentially metadata is just data! So we can use SQL to query the metadata. We access metadata table names via a prefix information_schema.. So for example we can query the table of all tables as information_schema.tables. We want to restrict it to tables in our default schema, "public", so we use a WHERE clause:

In [37]:
%%sql
SELECT * 
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.


[('imdb', 'public', 'actor_sample', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'cast_sample', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'movie_info_sample', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'cut1', 'VIEW', None, None, None, None, None, 'NO', 'NO', None),
 ('imdb', 'public', 'info_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'movie_sample', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'role_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('imdb', 'public', 'number_movies', 'VIEW', None, None, None, None, None, 'NO', 'NO', None)]

## Question 0
Now you write a query to get the name of all the tables in the PostgreSQL "information_schema" schema! 


In [38]:
%%sql result_0 <<
SELECT table_name 
from information_schema.tables
WHERE table_schema = 'information_schema';

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
69 rows affected.
Returning data to local variable result_0


In [39]:
# Do not delete/edit this cell
result_0.DataFrame().to_csv('results/result_0.csv', index=False)

In [40]:
result_0

[('information_schema_catalog_name',),
 ('attributes',),
 ('applicable_roles',),
 ('administrable_role_authorizations',),
 ('check_constraint_routine_usage',),
 ('character_sets',),
 ('check_constraints',),
 ('collations',),
 ('collation_character_set_applicability',),
 ('column_column_usage',),
 ('column_domain_usage',),
 ('routines',),
 ('column_privileges',),
 ('role_column_grants',),
 ('column_udt_usage',),
 ('columns',),
 ('constraint_column_usage',),
 ('routine_column_usage',),
 ('constraint_table_usage',),
 ('domain_constraints',),
 ('domain_udt_usage',),
 ('routine_privileges',),
 ('domains',),
 ('enabled_roles',),
 ('schemata',),
 ('key_column_usage',),
 ('role_routine_grants',),
 ('parameters',),
 ('referential_constraints',),
 ('routine_routine_usage',),
 ('routine_sequence_usage',),
 ('sequences',),
 ('routine_table_usage',),
 ('sql_parts',),
 ('sql_implementation_info',),
 ('sql_features',),
 ('sql_sizing',),
 ('table_constraints',),
 ('table_privileges',),
 ('view_table_usage',),
 ('role_table_grants',),
 ('tables',),
 ('transforms',),
 ('user_mappings',),
 ('triggered_update_columns',),
 ('views',),
 ('triggers',),
 ('udt_privileges',),
 ('foreign_data_wrappers',),
 ('role_udt_grants',),
 ('data_type_privileges',),
 ('usage_privileges',),
 ('role_usage_grants',),
 ('user_defined_types',),
 ('element_types',),
 ('view_column_usage',),
 ('view_routine_usage',),
 ('_pg_foreign_servers',),
 ('_pg_foreign_table_columns',),
 ('column_options',),
 ('_pg_foreign_data_wrappers',),
 ('foreign_table_options',),
 ('foreign_data_wrapper_options',),
 ('foreign_server_options',),
 ('foreign_servers',),
 ('_pg_foreign_tables',),
 ('user_mapping_options',),
 ('foreign_tables',),
 ('_pg_user_mappings',)]

In [41]:
grader.check("q0")

q0 results: All test cases passed!

# Question 1: Assessing Table Contents 
One of the first things you'll want to do with a database table is get a sense for its metadata: column names and types, and number of rows. 

We can use the PostgreSQL `\d` meta-command to get a description of all the columns in the `movie_info_sample` table.

In [42]:
%sql \d movie_info_sample

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
4 rows affected.


[['id', 'integer', ''],
 ['movie_id', 'integer', ''],
 ['info_type_id', 'integer', ''],
 ['info', 'character varying', '']]

## Question 1a
In the next cell, write an SQL query to calculate the number of rows in the `movie_info_sample` table.


In [43]:
%%sql result_1a <<

SELECT COUNT(*) from movie_info_sample;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_1a


In [44]:
result_1a = result_1a
result_1a

[(4866862,)]

Input the integer count from the query result into `count_1a`. In order to pass the autograder, please pass in the integer directly (e.g. `2000000`), and avoid extracting the integer from `result_1a`.

<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [45]:
count_1a = 2433431
count_1a

2433431

In [46]:
grader.check("q1a")

q1a results: All test cases passed!

### Taking a Random Sample (and Python variable substitution)

The cell positioned 2 below this one shows you a good way to take a random sample from a table: use the `TABLESAMPLE` clause after the table name in the `FROM` clause. It also shows you how you can reference a Python variable within a SQL string in the connection library we're using, `ipython-sql`. (The syntax for language-variable substitution is not part of the SQL standard, it's done by the connection library, so this syntax is specific to `ipython-sql`)

The SQL query below fetches a sample of size `p%` via the [Bernoulli sampling](https://wiki.postgresql.org/wiki/TABLESAMPLE_Implementation#BERNOULLI_Option) scheme, which instructs the database engine to "flip a coin" before deciding whether to return each tuple. 

## Question 1b
Given that you know the size of the table from the previous query, **write a formula to choose a sampling rate `p` that retrieves 5 tuples on expectation.** Your formula should contain `count_1a`. Don't forget to express `p` in units of percent (i.e. `p=5` is 5%)!

Try running the SQL cell many times and see what you notice.

<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [48]:
p_1b = 5/count_1a*100
p_1b

0.0002054712050598517

In [49]:
grader.check("q1b")

q1b results: All test cases passed!

In [50]:
%%sql
SELECT *
  FROM movie_info_sample TABLESAMPLE bernoulli('{p_1b}')

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


[(6869984, 2428861, 3, 'Action'),
 (6260939, 2055863, 3, 'Thriller'),
 (1648641, 2424383, 107, '$332,463 (USA) (26 September 2010)'),
 (9442307, 2004476, 1, '132'),
 (1372984, 1979304, 110, '31,247 (Spain) (7 December 2003)'),
 (5815246, 1761073, 3, 'Documentary'),
 (9601342, 2246874, 1, '54'),
 (4265080, 463944, 8, 'USA'),
 (9503528, 2097892, 1, '91'),
 (4435002, 1193338, 8, 'UK')]

## Question 1c

If you don't care about randomness, a more efficient way to get some arbitrary example rows from a table is to just use a `LIMIT` clause. In the next cell, fetch 5 rows from `movie_info_sample` using `LIMIT`.


In [51]:
%%sql result_1c <<
select * from  movie_info_sample limit 5;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.
Returning data to local variable result_1c


In [52]:
# Do not delete/edit this cell
result_1c.DataFrame().to_csv('results/result_1c.csv', index=False)

In [53]:
grader.check("q1c")

q1c results: All test cases passed!

# Question 2: Data Cleaning and Transformation

## Question 2a: MPAA Rating

The MPAA rating is something most datasets about movies contain and ours is no exception! But it's pretty messy to extract from the existing data. We're going to create a nice refined view of the `movie_sample` table that also includes a rating field.

### Question 2ai:
To start, using the `info_type` table, write a query to find which `id` corresponds to a film's MPAA rating.


In [10]:
%%sql result <<

select * from info_type;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
113 rows affected.
Returning data to local variable result


In [11]:
result.DataFrame().to_csv('results/result.csv', index=False)

In [12]:
result

[(1, 'runtimes'),
 (2, 'color info'),
 (3, 'genres'),
 (4, 'languages'),
 (5, 'certificates'),
 (6, 'sound mix'),
 (7, 'tech info'),
 (8, 'countries'),
 (9, 'taglines'),
 (10, 'keywords'),
 (11, 'alternate versions'),
 (12, 'crazy credits'),
 (13, 'goofs'),
 (14, 'soundtrack'),
 (15, 'quotes'),
 (16, 'release dates'),
 (17, 'trivia'),
 (18, 'locations'),
 (19, 'mini biography'),
 (20, 'birth notes'),
 (21, 'birth date'),
 (22, 'height'),
 (23, 'death date'),
 (24, 'spouse'),
 (25, 'other works'),
 (26, 'birth name'),
 (27, 'salary history'),
 (28, 'nick names'),
 (29, 'books'),
 (30, 'agent address'),
 (31, 'biographical movies'),
 (32, 'portrayed in'),
 (33, 'where now'),
 (34, 'trade mark'),
 (35, 'interviews'),
 (36, 'article'),
 (37, 'magazine cover photo'),
 (38, 'pictorial'),
 (39, 'death notes'),
 (40, 'LD disc format'),
 (41, 'LD year'),
 (42, 'LD digital sound'),
 (43, 'LD official retail price'),
 (44, 'LD frequency response'),
 (45, 'LD pressing plant'),
 (46, 'LD length'),
 (47, 'LD language'),
 (48, 'LD review'),
 (49, 'LD spaciality'),
 (50, 'LD release date'),
 (51, 'LD production country'),
 (52, 'LD contrast'),
 (53, 'LD color rendition'),
 (54, 'LD picture format'),
 (55, 'LD video noise'),
 (56, 'LD video artifacts'),
 (57, 'LD release country'),
 (58, 'LD sharpness'),
 (59, 'LD dynamic range'),
 (60, 'LD audio noise'),
 (61, 'LD color information'),
 (62, 'LD group genre'),
 (63, 'LD quality program'),
 (64, 'LD close captions-teletext-ld-g'),
 (65, 'LD category'),
 (66, 'LD analog left'),
 (67, 'LD certification'),
 (68, 'LD audio quality'),
 (69, 'LD video quality'),
 (70, 'LD aspect ratio'),
 (71, 'LD analog right'),
 (72, 'LD additional information'),
 (73, 'LD number of chapter stops'),
 (74, 'LD dialogue intellegibility'),
 (75, 'LD disc size'),
 (76, 'LD master format'),
 (77, 'LD subtitles'),
 (78, 'LD status of availablility'),
 (79, 'LD quality of source'),
 (80, 'LD number of sides'),
 (81, 'LD video standard'),
 (82, 'LD supplement'),
 (83, 'LD original title'),
 (84, 'LD sound encoding'),
 (85, 'LD number'),
 (86, 'LD label'),
 (87, 'LD catalog number'),
 (88, 'LD laserdisc title'),
 (89, 'screenplay-teleplay'),
 (90, 'novel'),
 (91, 'adaption'),
 (92, 'book'),
 (93, 'production process protocol'),
 (94, 'printed media reviews'),
 (95, 'essays'),
 (96, 'other literature'),
 (97, 'mpaa'),
 (98, 'plot'),
 (99, 'votes distribution'),
 (100, 'votes'),
 (101, 'rating'),
 (102, 'production dates'),
 (103, 'copyright holder'),
 (104, 'filming dates'),
 (105, 'budget'),
 (106, 'weekend gross'),
 (107, 'gross'),
 (108, 'opening weekend'),
 (109, 'rentals'),
 (110, 'admissions'),
 (111, 'studios'),
 (112, 'top 250 rank'),
 (113, 'bottom 10 rank')]

In [13]:
%%sql result_2ai <<

select id from info_type where info = 'mpaa';

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_2ai


In [14]:
result_2ai = result_2ai
result_2ai

[(97,)]

Input the id you found into `mpaa_rating_id`. Again, in order to pass the autograder, please pass in the integer id directly (e.g. `50`).

<!--
BEGIN QUESTION
name: q2ai
points: 1
-->

In [15]:
mpaa_rating_id = 97
mpaa_rating_id

97

In [16]:
grader.check("q2ai")

q2ai results: All test cases passed!

You may use `mpaa_rating_id` directly in the next question using python variable substitution.

### Question 2aii:
In the next cell,
- Construct a view named `movie_ratings` containing one row for each movie containing `(movie_id, title, info, mpaa_rating)`, where `info` is the full MPAA rating string from `movie_info_sample`, and `mpaa_rating` is just the rating itself (i.e. `R`, `PG-13`, `PG`, etc).
- Following the view definition, write a `SELECT` query to return the first 20 rows of the view ordered by ascending `movie_id`.

**Before you get started**, take a look at the `movie_info_sample` tuples corresponding to the MPAA rating. The `info` field is a little longer than just the rating. It includes an explanation for why that movie received its score. You will need to extract a substring from the `info` column of `movie_info_sample`; you can use the [string functions](https://www.postgresql.org/docs/current/functions-string.html) in PostgreSQL to do it. There are many possible solutions. One possible solution is to use the substring function along with regex. If you use this approach, [this section on regex](https://www.postgresql.org/docs/current/functions-matching.html#FUNCTIONS-POSIX-REGEXP) may be particularly useful. [regex101.com](https://regex101.com) may also be helpful to craft your regular expressions.

In [17]:
%%sql movie_sample <<

select * from movie_sample limit 20;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
20 rows affected.
Returning data to local variable movie_sample


In [18]:
movie_sample.DataFrame().to_csv('results/sample.csv', index=False)

In [19]:
movie_sample

[(2038405, 'La corte de faraón', 1944),
 (2081186, 'Long de xin', 1985),
 (2177749, 'Onésime aime les bêtes', 1913),
 (1718608, 'Bedtime Worries', 1933),
 (2130699, 'Mothman', 2000),
 (2415354, 'The Spiral Staircase', 1945),
 (2317538, 'Struck by Lightning', 2012),
 (1963718, 'Humoresque', 1946),
 (2001116, 'Jingle All The Way', 2012),
 (2186697, 'Pajama Party', 1964),
 (1675784, 'All Men Are Apes!', 1965),
 (2391276, 'The Man from Montana', 1917),
 (2459326, 'Ulterior Motives', 2008),
 (1667713, 'Africa: Texas Style', 1967),
 (2239107, 'Rendez-vous à Paris', 1947),
 (2018134, 'Kickboxer 2: The Road Back', 1991),
 (1648449, 'A Favorite Fool', 1915),
 (1778514, 'Claude', 1993),
 (1814171, 'Deep River: The Island', 2009),
 (1743296, 'Boyle Heights', 2010)]

In [23]:
%%sql sample <<

select * from movie_info_sample where movie_id = 1632926;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
6 rows affected.
Returning data to local variable sample


In [24]:
sample.DataFrame().to_csv('results/sample.csv', index=False)

In [25]:
sample

[(9200179, 1632926, 1, 'Canada:90'),
 (9200180, 1632926, 1, 'USA:98'),
 (2043233, 1632926, 97, 'Rated PG-13 for sexual content, brief nudity and language'),
 (4542081, 1632926, 8, 'USA'),
 (5610541, 1632926, 3, 'Comedy'),
 (5610542, 1632926, 3, 'Drama')]

In [38]:
%%sql result_2aii <<

DROP VIEW IF EXISTS movie_ratings;
CREATE VIEW movie_ratings AS

(select t1.movie_id, t2.title, t1.info, (regexp_match(t1.info,'PG-13 |PG |R '))[1] as mpaa_rating
 from  movie_info_sample t1
 inner join movie_sample t2 on t1.movie_id = t2.id where t1.info_type_id = 97);

select * from movie_ratings order by movie_id limit 20;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
20 rows affected.
Returning data to local variable result_2aii


In [39]:
# Do not delete/edit this cell
result_2aii.DataFrame().to_csv('results/result_2aii.csv', index=False)

In [40]:
grader.check("q2aii")

q2aii results: All test cases passed!

## Question 2b - Movie Moola

One measure of a movie's success is how much money it makes. If we look at our `info_type` table, we have information about the film's gross earnings and the budget for a film. It would be nice to know how much money a film made using the profit formula:
$$profit = earnings - moneyspent$$

We start by taking a look at the gross info type, with `info_type_id = 107`.

In [94]:
%%sql
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 107
ORDER BY id
LIMIT 10 OFFSET 100000;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


[(1321635, 2355233, 107, '£554,314 (UK) (17 September 2000)'),
 (1321635, 2355233, 107, '£554,314 (UK) (17 September 2000)'),
 (1321636, 2355233, 107, '$42,821,784 (Worldwide) (except USA)'),
 (1321636, 2355233, 107, '$42,821,784 (Worldwide) (except USA)'),
 (1321637, 2355233, 107, '$104,155,843 (Worldwide)'),
 (1321637, 2355233, 107, '$104,155,843 (Worldwide)'),
 (1321638, 2355233, 107, 'AUD 1,890,495 (Australia) (14 December 2000)'),
 (1321638, 2355233, 107, 'AUD 1,890,495 (Australia) (14 December 2000)'),
 (1321639, 2355233, 107, '€ 3,974,514 (Spain) (10 August 2002)'),
 (1321639, 2355233, 107, '€ 3,974,514 (Spain) (10 August 2002)')]

There are a lot of things to notice here. First of all, the `info` is a string with not only the earnings but the country and the month the earnings are cummulatively summed till. Additionally, the info is not in the same currency! On top of that, it looks like some of the gross earnings even for those in USD are from worldwide sales while others only count sales within the USA.

For consistency, let's only use movies with gross earnings counted in the USA, and in US Dollars ($). 

### Question 2bi:
We want the number part of the `info` column and the **latest (maximum) earnings value** for a particular film. In the next cell,
- Construct a view named `movie_gross` containing one row for each movie containing `(gross, movie_id, title)`, where `gross` is the dollar amount extracted as a float.
- To take a look at our cleaned data, write a `SELECT` query to display the top 10 highest grossing films from `movie_gross`.

**Hint:** We extracted the rating above just like how we want to isolate the dollar amount as a string (There are multiple ways of doing this).

**Hint:** Look at the [regexp_replace](https://www.postgresql.org/docs/9.4/functions-matching.html) function and the 'g' tag

In [16]:
%%sql result_2bi <<

DROP VIEW IF EXISTS movie_gross;
CREATE VIEW movie_gross AS

(SELECT MAX(CAST(regexp_replace(substring(t1.info from '%$#"[0-9,]+#"%' for '#'),',','','g') AS float)) as gross,t1.movie_id, t2.title
FROM movie_info_sample t1
INNER join movie_sample t2 
ON t1.movie_id = t2.id  
WHERE t1.info_type_id = 107 AND t1.info LIKE '$%' AND t1.info LIKE '%(USA)%'
group by t1.movie_id, t2.title);

select* from movie_gross

ORDER BY gross DESC limit 10;



 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
10 rows affected.
Returning data to local variable result_2bi


In [17]:
# Do not delete/edit this cell
result_2bi.DataFrame().to_csv('results/result_2bi.csv', index=False)

In [18]:
result_2bi

[(760507625.0, 1704289, 'Avatar'),
 (658672302.0, 2438179, 'Titanic'),
 (623357910.0, 2346436, 'The Avengers'),
 (534858444.0, 2360583, 'The Dark Knight'),
 (460935665.0, 2310522, 'Star Wars'),
 (448139099.0, 2360588, 'The Dark Knight Rises'),
 (436471036.0, 2285018, 'Shrek 2'),
 (435110554.0, 1851357, 'E.T. the Extra-Terrestrial'),
 (431065444.0, 2310573, 'Star Wars: Episode I - The Phantom Menace'),
 (423315812.0, 2204345, "Pirates of the Caribbean: Dead Man's Chest")]

In [19]:
grader.check("q2bi")

q2bi results: All test cases passed!

We now take a look at the budget info type, with `info_type_id = 105`.

In [20]:
%%sql
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 105
ORDER BY id
LIMIT 10 OFFSET 5000;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


[(1261074, 1983149, 105, '$75,000,000'),
 (1261110, 1983269, 105, 'INR 180,000,000'),
 (1261160, 2381188, 105, '$40,000,000'),
 (1261170, 1991083, 105, 'FIM 9,219,499'),
 (1261210, 1993907, 105, '$45,000,000'),
 (1261247, 1995787, 105, '$38,000,000'),
 (1261308, 1999081, 105, '$50,000,000'),
 (1261324, 1999196, 105, 'SEK 40,000,000'),
 (1261375, 2001114, 105, '$60,000,000'),
 (1261396, 2001989, 105, '$13,000,000')]

Similar to when we examined the gross info, we see a lot of non-US dollar currencies. For consistency, let's only use movies with budget in US dollars.

### Question 2bii:

Now, we want something similar for the budget of the film so we can perform the subtraction of the `gross` and `budget`. We want the number part of the `info` column and the **maximum budget value** for a particular film (as you can verify, some movies have more than one budget). In the next cell,
- Construct a view named `movie_budget` containing one row for each movie containing `(budget, movie_id, title)`, where `budget` is the dollar amount extracted as a float.
- To take a look at our cleaned data, write a `SELECT` query to display the top 10 highest budget films from `movie_budget`. When multiple films have the same budget, break tie by `movie_id` (ascending).

**Hint:** The query here should be quite similar to q2bi. Make sure to break ties properly!

In [21]:
%%sql result_2bii <<

DROP VIEW IF EXISTS movie_budget;
CREATE VIEW movie_budget AS
(SELECT MAX(CAST(regexp_replace(substring(t1.info from '%$#"[0-9,]+#"%' for '#'),',','','g') AS float)) as budget,t1.movie_id, t2.title
FROM movie_info_sample t1
INNER join movie_sample t2 
ON t1.movie_id = t2.id  
WHERE t1.info_type_id = 105 AND t1.info LIKE '$%'
group by t1.movie_id, t2.title);

select* from movie_budget

ORDER BY budget DESC, movie_id ASC limit 10;


 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
10 rows affected.
Returning data to local variable result_2bii


In [22]:
# Do not delete/edit this cell
result_2bii.DataFrame().to_csv('results/result_2bii.csv', index=False)

In [23]:
result_2bii

[(300000000.0, 2204343, "Pirates of the Caribbean: At World's End"),
 (260000000.0, 2332419, 'Tangled'),
 (258000000.0, 2305993, 'Spider-Man 3'),
 (250000000.0, 1938937, 'Harry Potter and the Half-Blood Prince'),
 (250000000.0, 2002374, 'John Carter'),
 (250000000.0, 2204347, 'Pirates of the Caribbean: On Stranger Tides'),
 (250000000.0, 2360588, 'The Dark Knight Rises'),
 (250000000.0, 2387922, 'The Lone Ranger'),
 (237000000.0, 1704289, 'Avatar'),
 (230000000.0, 2344435, 'The Amazing Spider-Man')]

In [24]:
grader.check("q2bii")

q2bii results: All test cases passed!

### Question 2biii:

We have all the parts we need to calculate the profits. Using the `movie_gross` and `movie_budget` views created above, we can subtract the numeric columns and save the result in another column called `profit`.

In the next cell, construct a view named `movie_profit` containing one row for each movie containing `(movie_id, title, profit)`, where `profit` is the result of subtracting that movie's `budget` from `gross`. Following the view definition, write a `SELECT` query to return the first 10 rows of the view ordered by descending `profit`. This may take a second.

In [25]:
%%sql result_2biii <<

DROP VIEW IF EXISTS movie_profit;
CREATE VIEW movie_profit AS

(SELECT t1.movie_id, t1.title, (t1.gross - t2.budget) as profit
 from movie_gross t1
 INNER JOIN movie_budget t2 ON t1.movie_id = t2.movie_id);


SELECT * from movie_profit ORDER BY profit DESC limit 10;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
10 rows affected.
Returning data to local variable result_2biii


In [26]:
# Do not delete/edit this cell
result_2biii.DataFrame().to_csv('results/result_2biii.csv', index=False)

In [27]:
result_2biii

[(1704289, 'Avatar', 523507625.0),
 (2438179, 'Titanic', 458672302.0),
 (2310522, 'Star Wars', 449935665.0),
 (1851357, 'E.T. the Extra-Terrestrial', 424610554.0),
 (2346436, 'The Avengers', 403357910.0),
 (2360583, 'The Dark Knight', 349858444.0),
 (2400712, 'The Passion of the Christ', 340782930.0),
 (2006991, 'Jurassic Park', 338820792.0),
 (2172509, 'Olympus Has Fallen', 330824682.0),
 (2379293, 'The Hunger Games', 330010692.0)]

In [28]:
grader.check("q2biii")

q2biii results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 2biv:

We analyzed the data but something seems odd. On a closer look, there are many negative values for `profit`. For example, the movie `102 Dalmations` looks to have lost ~$18M but it was a widely successful film! What may account for this issue?

<!--
BEGIN QUESTION
name: q2biv
manual: true
points: 1
-->

The movie eanring in this question only account for the domestic earning. There are oversea earnings that needed to be consider when calculating profit.

<!-- END QUESTION -->



# Question 3: Using Cleaned Data

Now that we have gone through all the work of cleaning our financials from `info` in `movie_info_sample`, let's take a closer look at the data we generated. 

### Question 3a: Earnings per Genre

Another `info_type` available to us is the movie genre. Looking at the `movie_gross` values, how much does each genre earn on average from the US?

- Create a view with the columns `movie_id`, `title`, `gross`, `genre`, and `average_genre` where `gross` is a movie's gross US earnings, `genre` is the movie's genre, and `average_genre` is the average earnings for the corresponding genre. If a movie has multiple genres, the movie should appear in multiple rows with each genre as a row.

- Following the view definition, write a `SELECT` query to return the rows for the movie "Mr. & Mrs. Smith" ordered by genre alphabetically.

**Hint:** Look into [window functions](https://www.postgresql.org/docs/9.1/tutorial-window.html)


In [29]:
%%sql result_3a <<

DROP VIEW IF EXISTS movie_avg_genre;
CREATE VIEW movie_avg_genre AS

(SELECT t1.movie_id, t1.title, t1.gross, t2.info as genre, (AVG(gross) OVER (PARTITION BY t2.info)) as average_genre
FROM movie_gross t1
INNER JOIN movie_info_sample as t2 
on t1.movie_id = t2.movie_id WHERE t2.info_type_id =3);

select * from movie_avg_genre where title = 'Mr. & Mrs. Smith' order by genre ;


 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
3 rows affected.
Returning data to local variable result_3a


In [30]:
# Do not delete/edit this cell
result_3a.DataFrame().to_csv('results/result_3a.csv', index=False)

In [31]:
result_3a

[(2132092, 'Mr. & Mrs. Smith', 186336103.0, 'Action', 42123826.131625965),
 (2132092, 'Mr. & Mrs. Smith', 186336103.0, 'Comedy', 21583843.81801513),
 (2132092, 'Mr. & Mrs. Smith', 186336103.0, 'Romance', 18470817.081399772)]

In [32]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b: Analyzing Gross Earnings

A good way to view numerical data is through a histogram. The histogram shows the spread of the data along with several other key attributes that allow for the data to be analyzed. 

We went through a lot of work transforming the gross earnings `info` string into a numerical value. Because of our hard work, we can now look more closely at this data and understand what it looks like. We will generate a [five-number summary](https://en.wikipedia.org/wiki/Five-number_summary) and the average of the US gross earnings data to do this.

- Create a view named `earnings_summary` of a one row summary of the `movie_gross` `gross` data with the `min`, `25th_percentile`, `median`, `75th_percentile`, `max`, and `average`. 
- Following the view definition, write a `SELECT` query to display it.

**Hint:** Look at SQL [aggregate functions](https://www.postgresql.org/docs/9.4/functions-aggregate.html). You may find some useful.


In [52]:
%%sql result_3b <<

DROP VIEW IF EXISTS earnings_summary;
CREATE VIEW earnings_summary AS

(SELECT min(gross) , 
(percentile_cont(0.25) WITHIN GROUP (ORDER BY gross)) as "25th_percentile", 
(percentile_cont(0.50) WITHIN GROUP (ORDER BY gross)) as median, 
(percentile_cont(0.75) WITHIN GROUP (ORDER BY gross)) as "75th_percentile",
max(gross), 
AVG(gross) as average
FROM movie_gross);

select * from  earnings_summary;


 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
1 rows affected.
Returning data to local variable result_3b


In [53]:
# Do not delete/edit this cell
result_3b.DataFrame().to_csv('results/result_3b.csv', index=False)

In [54]:
result_3b

[(30.0, 166623.0, 2317091.0, 20002717.5, 760507625.0, 19594424.63641884)]

In [55]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3c

What do you notice about the summary values generated in `earnings_summary`? Identify two properties about the histogram of the data.

**Hint:** Think in terms of about concepts from statistics like spread, modality, skew, etc. and how they may apply here.

<!--
BEGIN QUESTION
name: q3c
manual: true
points: 1
-->

The average is much greater than the mean, which suggests it skew right. The min is 30 and the max is 760507625, suggesting that the spread is dispersed

<!-- END QUESTION -->



# Question 4: Joins

Joins are a powerful tool in database cleaning and analysis. They allow for the user to create useful tables and bring together information in a meaningful way. 

There are many types of joins: inner, outer, left, right, etc. Let's practice these in a special scenario. 

You are now working as a talent director and you need a list of all `actor` roles and the number of movies in which they have acted. 

- Create a view called `number_movies` which columns `id`, `name`, `number` where `id` is the actor's id, `name` is the actor's name, and `number` is the number of movies they have acted in. '
- Following your view, write a ``SELECT`` query to display the top 10 actors who have been in the most films.

**Note:** The `cast_sample` may include actors not included in `actor_sample` table. We still want to include these actors in our result by reference to their id. The `name` field can be NULL.

In [56]:
%%sql cast <<
select * from cast_sample limit 20;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
20 rows affected.
Returning data to local variable cast


In [57]:
cast.DataFrame().to_csv('results/cast.csv', index=False)
cast

[(708, 235, 2345369, 1),
 (721, 241, 2504309, 1),
 (789, 264, 2156734, 1),
 (875, 299, 1954994, 1),
 (888, 302, 765037, 1),
 (889, 302, 765172, 1),
 (898, 306, 291387, 1),
 (899, 306, 1477434, 1),
 (931, 324, 824119, 1),
 (1936, 543, 1754068, 1),
 (1937, 543, 1874542, 1),
 (1938, 543, 1998895, 1),
 (1939, 543, 2011539, 1),
 (1940, 543, 2217855, 1),
 (2558, 634, 2515763, 1),
 (2718, 696, 727052, 1),
 (2771, 717, 2387737, 1),
 (3570, 767, 1813776, 1),
 (3571, 767, 1949302, 1),
 (3572, 767, 162704, 1)]

In [58]:
%%sql actor <<
select * from actor_sample limit 20;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
20 rows affected.
Returning data to local variable actor


In [59]:
actor.DataFrame().to_csv('results/actor.csv', index=False)
actor

[(2591445, 'Taylor, Joan', 'f'),
 (640876, 'Harris, R.H.', 'm'),
 (614937, 'Gélinas, Jean-Maurice', 'm'),
 (987924, 'Martins, Pedro', 'm'),
 (104379, 'Bass, Monty', 'm'),
 (66819, 'Atanasov, Grudi', 'm'),
 (1309781, 'Rios, Raul', 'm'),
 (497885, 'Flouw, Jonathan', 'm'),
 (484858, 'Ficarra, Amedeo', 'm'),
 (2356357, 'Nance, Nichole', 'f'),
 (1850956, 'Bryant, Lela', 'f'),
 (463041, 'Evans, Troy', 'm'),
 (842805, 'Kravitz, Steven', 'm'),
 (1763469, 'Andreasson, Ann-Katrin', 'f'),
 (70838, 'Austin, Matt', 'm'),
 (2444621, 'Rain, Mykal', 'f'),
 (261751, 'Champagne, Jean', 'm'),
 (2212104, 'Lafleur, Jennifer', 'f'),
 (1707507, 'Xiong, Matthew', 'm'),
 (66815, 'Atanasov, Dimitre', 'm')]

In [21]:
%%sql result_4 <<

DROP VIEW IF EXISTS number_movies;
CREATE VIEW number_movies AS 

SELECT group_table.id, name, COUNT(*) as number from

(SELECT a.id, a.name, c.movie_id as movie from cast_sample c
left JOIN actor_sample a
ON c.person_id = a.id
where c.role_id =1) as group_table



GROUP BY group_table.id, name ;

select * from number_movies ORDER BY number DESC limit 10;

 * postgresql://jovyan@127.0.0.1:5432/imdb
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
10 rows affected.
Returning data to local variable result_4


In [22]:
# Do not delete/edit this cell
result_4.DataFrame().to_csv('results/result_4.csv', index=False)

In [23]:
result_4

[(95397, 'Barker, Bob', 6853),
 (515315, 'Freeman, Morgan', 5938),
 (677696, 'Hinnant, Skip', 4697),
 (1573853, 'Trebek, Alex', 4690),
 (1362169, 'Sajak, Pat', 3937),
 (1417394, 'Shaffer, Paul', 3546),
 (911160, 'Lima, Pedro', 2911),
 (900749, 'Letterman, David', 2895),
 (487253, 'Filipe, Guilherme', 2861),
 (356575, 'Davidson, Doug', 2760)]

In [24]:
grader.check("q4")

q4 results: All test cases passed!

# Congratulations! You have finished Project 1.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**For submission on Gradescope, you will need to submit BOTH the proj1.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 1 directory within JupyterHub (remove `proj1.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [25]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

  adding: results/ (stored 0%)
  adding: results/result_0.csv (deflated 66%)
  adding: results/result_1c.csv (deflated 28%)
  adding: results/result.csv (deflated 47%)
  adding: results/sample.csv (deflated 31%)
  adding: results/result_2aii.csv (deflated 61%)
  adding: results/result_2bi.csv (deflated 31%)
  adding: results/result_2bii.csv (deflated 42%)
  adding: results/result_2biii.csv (deflated 30%)
  adding: results/result_3a.csv (deflated 40%)
  adding: results/result_3b.csv (deflated 23%)
  adding: results/cast.csv (deflated 49%)
  adding: results/actor.csv (deflated 32%)
  adding: results/result_4.csv (deflated 25%)


/home/jovyan/fa22/proj/proj1/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [47]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1b results:
    q1b - 1 result:
        Trying:
            0.0001 <= p_1b <= 0.001
        Expecting:
            True
        **********************************************************************
        Line 1, in q1b 0
        Failed example:
            0.0001 <= p_1b <= 0.001
        Exception raised:
            Traceback (most recent call last):
              File "/opt/conda/lib/python3.9/doctest.py", line 1334, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1b 0[0]>", line 1, in <module>
                0.0001 <= p_1b <= 0.001
            NameError: name 'p_1b' is not defined

q1c results: All test cases passed!

q2ai results:
    q2ai - 1 result:
        Trying:
            mpaa_rating_id == 97
        Expecting:
            True
        **********************************************************************
        Line 1, in q2ai 0
        Fail

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()